In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pickle
import warnings

In [ ]:
IIRF_HORIZON=100
IIRF_MAX = 99.95
GAS_BOX_AXIS=4
TIME_AXIS=0
M_ATMOS = 5.1352e18 # mass of atmosphere, kg

In [ ]:
scenarios = ['abrupt-4xCO2']
n_scenarios = len(scenarios)

In [ ]:
timebounds = np.arange(1750, 2501.5, 1)
timesteps  = np.arange(1750.5, 2501, 1)
timestep   = 1

In [ ]:
n_timesteps = len(timesteps)
n_timebounds = len(timebounds)

In [ ]:
gas_list = ["CO2"]
n_gases = len(gas_list)

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for scenario in scenarios:
    concentration_rcmip[scenario] = {}
    for gas in gas_list:
        gas_rcmip_name = gas.replace("-", "")
        concentration_rcmip[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.squeeze()

In [ ]:
concentration_rcmip['abrupt-4xCO2']['CO2'][:100]=concentration_rcmip['abrupt-4xCO2']['CO2'][100]/4
concentration_rcmip['abrupt-4xCO2']['CO2'] = np.insert(concentration_rcmip['abrupt-4xCO2']['CO2'], 0, concentration_rcmip['abrupt-4xCO2']['CO2'][0], axis=0)
concentration_rcmip['abrupt-4xCO2']['CO2']

In [ ]:
concentration_array = concentration_rcmip['abrupt-4xCO2']['CO2'].reshape((752, 1, 1, 1, 1))

In [ ]:
N_GAS_BOXES=4

partition_fraction = {}
partition_fraction["CO2"] = np.array([0.2173, 0.2240, 0.2824, 0.2763])

lifetime = {
    "CO2": np.array([1e9, 394.4, 36.54, 4.304]),
}

MOLWT = {
    "AIR": 28.97,  # reference?
    "C": 12.011,
    "CO2": 44.009,
}

iirf_0 = {}
iirf_0['CO2'] = 29

iirf_uptake = {}
iirf_uptake["CO2"] = 0.00846

iirf_temperature = {}
#iirf_temperature = {gas: -iirf_0[gas]*0.015 for gas in gas_list}
iirf_temperature["CO2"] = 4.0

iirf_airborne = {}
iirf_airborne["CO2"] = 0.000819

concentration_per_emission = {}
for gas in gas_list:
    concentration_per_emission[gas] = (
        1 / (M_ATMOS / 1e18 * MOLWT[gas] / MOLWT["AIR"])
    )

rcmip_emissions_adjustment = {}
for gas in gas_list:
    rcmip_emissions_adjustment[gas] = 0

In [ ]:
lifetime_array = np.ones((1, 1, n_gases, 1, N_GAS_BOXES)) * np.nan
partition_fraction_array = np.ones((1, 1, n_gases, 1, N_GAS_BOXES)) * np.nan
iirf_0_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_airborne_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_temperature_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_uptake_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
concentration_per_emission_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
rcmip_emissions_adjustment_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan  # could be time dependent
baseline_concentration_array = concentration_array[0:1, :, :, :, :]
emissions_array = np.ones((n_timesteps, n_scenarios, n_gases, 1, 1)) * np.nan

# we would also loop/parallel over config here
for igas, gas in enumerate(gas_list):
    lifetime_array[0, 0, igas, 0, :] = lifetime[gas]
    partition_fraction_array[0, 0, igas, 0, :] = partition_fraction[gas]
    iirf_0_array[0, 0, igas, 0, :] = iirf_0[gas]
    iirf_airborne_array[0, 0, igas, 0, :] = iirf_airborne[gas]
    iirf_temperature_array[0, 0, igas, 0, :] = iirf_temperature[gas]
    iirf_uptake_array[0, 0, igas, 0, :] = iirf_uptake[gas]
    concentration_per_emission_array[0, 0, igas, 0, 0] = concentration_per_emission[gas]
    rcmip_emissions_adjustment_array[0, 0, igas, 0, 0] = rcmip_emissions_adjustment[gas]

In [ ]:
g1 = np.sum(
    partition_fraction_array * lifetime_array *
    (1 - (1 + IIRF_HORIZON/lifetime_array) *
    np.exp(-IIRF_HORIZON/lifetime_array)), axis=GAS_BOX_AXIS, keepdims=True
)

g0 = np.exp(
    -1 * np.sum(
        partition_fraction_array*lifetime_array*(1 - np.exp(-IIRF_HORIZON/lifetime_array)), axis=4, keepdims=True
    )/g1
)

In [ ]:
g1[0,0,0,0,0], g0[0,0,0,0,0]

In [ ]:
alpha_lifetime_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan
airborne_emissions_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan
gas_boxes_array = np.zeros((1, n_scenarios, n_gases, 1, N_GAS_BOXES))  # initial condition for restarts

emissions_array = np.ones((n_timesteps, n_scenarios, n_gases, 1, 1)) * np.nan
cumulative_emissions_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan
cumulative_emissions_array[0, ...] = 0  # initial condition: make interface option
#cumulative_emissions_array[1:, ...] = np.cumsum(emissions_array * timestep, axis=TIME_AXIS)

airborne_emissions_array[0, ...] = 0  # initial condition: make interface option

In [ ]:
def calculate_alpha(
    cumulative_emissions,
    airborne_emissions,
    temperature,
    iirf_0,
    iirf_uptake,
    iirf_temperature,
    iirf_airborne,
    g0,
    g1,
    iirf_max,
):

    iirf = iirf_0 + iirf_uptake * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)

    # overflow and invalid value errors occur with very large and small values
    # in the exponential. This happens with very long lifetime GHGs. Usually
    # these GHGs don't have a temperature dependence on IIRF but even if they
    # did the lifetimes are so long that it is unlikely to have an effect.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        alpha = g0 * np.exp(iirf / g1)
        alpha[np.isnan(alpha)]=1
    return alpha

In [ ]:
def unstep_concentration(
    concentration,
    gas_boxes_old,
    airborne_emissions_old,
    concentration_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    baseline_concentration,
    timestep=1,
    natural_emissions_adjustment=0,
):

    # comments are keeping track of units
    decay_rate = timestep/(alpha_lifetime * lifetime)   # [1]
    decay_factor = np.exp(-decay_rate)  # [1]

    # [kg] = [ppm] - [ppm] / [ppm/kg]
    airborne_emissions_new = (concentration-baseline_concentration)/concentration_per_emission

    # [kg/yr] = [kg] - [kg]*[1] / ([1] * [1] * [1] * [yr])
    emissions = (
        (airborne_emissions_new - np.sum(gas_boxes_old*decay_factor, axis=GAS_BOX_AXIS, keepdims=True)) /
        (np.sum(
            partition_fraction / decay_rate * ( 1. - decay_factor ) * timestep,
            axis=GAS_BOX_AXIS, keepdims=True)
        )
    )

    # [kg] = ([yr] * [kg/yr] * [1] / [1] * [1]) + [kg] * [1]
    gas_boxes_new = timestep * emissions * partition_fraction * 1/decay_rate * ( 1. - decay_factor ) + gas_boxes_old * decay_factor
    emissions_out = emissions + natural_emissions_adjustment

    return emissions_out, gas_boxes_new, airborne_emissions_new

In [ ]:
for i_timestep in range(n_timesteps):  # 0 to 751
    alpha_lifetime_array[i_timestep, ...] = calculate_alpha(   # this timestep
        cumulative_emissions_array[i_timestep, ...],  # last timebound
        airborne_emissions_array[i_timestep, ...],  # last timebound
        np.zeros((1, 1, 1, 1)),  # last timebound
        iirf_0_array,
        iirf_uptake_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
        IIRF_MAX
    )
    
    emissions_array[i_timestep, ...], gas_boxes_array, airborne_emissions_array[i_timestep+1, ...] = unstep_concentration( # this timestep 
        concentration_array[i_timestep+1, :], # next timebound
        gas_boxes_array, # last timebound
        airborne_emissions_array[i_timestep, ...], # last timebound
        concentration_per_emission_array,
        lifetime_array,
        alpha_lifetime_array[i_timestep, ...], # last timebound
        partition_fraction_array,
        baseline_concentration_array,
        timestep,
        rcmip_emissions_adjustment_array,
    )
    cumulative_emissions_array[i_timestep+1, ...] = cumulative_emissions_array[i_timestep, ...] + emissions_array[i_timestep, ...] * timestep

In [ ]:
pl.plot(np.arange(1750.5, 2501), emissions_array[:, 0, 0, 0, 0])

In [ ]:
pl.plot(np.arange(1750.5, 2501), emissions_array[:, 0, 0, 0, 0])
pl.xlim(1860, 2500)
pl.ylim(0, 50)

In [ ]:
pl.plot(np.arange(1750, 2502), cumulative_emissions_array[:, 0, 0, 0, 0])